### Глава 14

In [1]:
import sys, random, math
from collections import Counter
import numpy as np
import sys

np.random.seed(0)

raw = ''
with open('moscow_petushki.txt') as f:
    for r in f.readlines():
        raw += r # получаем одну большую строку

vocab = list(set(raw)) # получаем список уникальных символов
word2index = {}
for i, word in enumerate (vocab):
    word2index[word] = i # каждому символу присваиваем индекс
indices = np.array(list(map(lambda x: word2index[x], raw) ) ) # зашифровываем весь текст номерами символов
print(indices) # получаем одномерный массив из почти 100 тыс номеров символов

[60 59 68 ... 28 39  8]


In [2]:
import numpy as np

class Tensor (object):
    
    def __init__(self,data,
                 autograd=False,
                 creators=None,
                 creation_op=None,
                 id=None):
        
        self.data = np.array(data)
        self.autograd = autograd
        self.grad = None
        if(id is None):
            self.id = np.random.randint(0,100000)
        else:
            self.id = id
        
        self.creators = creators
        self.creation_op = creation_op
        self.children = {}
        
        if(creators is not None):
            for c in creators:
                if(self.id not in c.children):
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1

    def all_children_grads_accounted_for(self):
        for id,cnt in self.children.items():
            if(cnt != 0):
                return False
        return True 
        
    def backward(self,grad=None, grad_origin=None):
        if(self.autograd):
 
            if(grad is None):
                grad = Tensor(np.ones_like(self.data))

            if(grad_origin is not None):
                if(self.children[grad_origin.id] == 0):
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1

            if(self.grad is None):
                self.grad = grad
            else:
                self.grad += grad
            
            assert grad.autograd == False
            
            if(self.creators is not None and 
               (self.all_children_grads_accounted_for() or 
                grad_origin is None)):

                if(self.creation_op == "add"):
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)
                    
                if(self.creation_op == "sub"):
                    self.creators[0].backward(Tensor(self.grad.data), self)
                    self.creators[1].backward(Tensor(self.grad.__neg__().data), self)

                if(self.creation_op == "mul"):
                    new = self.grad * self.creators[1]
                    self.creators[0].backward(new , self)
                    new = self.grad * self.creators[0]
                    self.creators[1].backward(new, self)                    
                    
                if(self.creation_op == "mm"):
                    c0 = self.creators[0]
                    c1 = self.creators[1]
                    new = self.grad.mm(c1.transpose())
                    c0.backward(new)
                    new = self.grad.transpose().mm(c0).transpose()
                    c1.backward(new)
                    
                if(self.creation_op == "transpose"):
                    self.creators[0].backward(self.grad.transpose())

                if("sum" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    self.creators[0].backward(self.grad.expand(dim,
                                                               self.creators[0].data.shape[dim]))

                if("expand" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    self.creators[0].backward(self.grad.sum(dim))
                    
                if(self.creation_op == "neg"):
                    self.creators[0].backward(self.grad.__neg__())
                    
                if(self.creation_op == "sigmoid"):
                    ones = Tensor(np.ones_like(self.grad.data))
                    self.creators[0].backward(self.grad * (self * (ones - self)))
                    
                if(self.creation_op == "tanh"):
                    ones = Tensor(np.ones_like(self.grad.data))
                    self.creators[0].backward(self.grad * (ones - (self * self)))
                
                if(self.creation_op == "relu"):
                    ones = Tensor((self.grad.data > 0))
                    self.creators[0].backward(self.grad * (ones * self ))
                
                if(self.creation_op == "index_select"):
                    new_grad = np.zeros_like(self.creators[0].data)
                    indices_ = self.index_select_indices.data.flatten()
                    grad_ = grad.data.reshape(len(indices_), -1)
                    for i in range(len(indices_)):
                        new_grad[indices_[i]] += grad_[i]
                    self.creators[0].backward(Tensor(new_grad))
                    
                if(self.creation_op == "cross_entropy"):
                    dx = self.softmax_output - self.target_dist
                    self.creators[0].backward(Tensor(dx))
                    
    def __add__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data + other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="add")
        return Tensor(self.data + other.data)

    def __neg__(self):
        if(self.autograd):
            return Tensor(self.data * -1,
                          autograd=True,
                          creators=[self],
                          creation_op="neg")
        return Tensor(self.data * -1)
    
    def __sub__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data - other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="sub")
        return Tensor(self.data - other.data)
    
    def __mul__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(self.data * other.data,
                          autograd=True,
                          creators=[self,other],
                          creation_op="mul")
        return Tensor(self.data * other.data)    

    def sum(self, dim):
        if(self.autograd):
            return Tensor(self.data.sum(dim),
                          autograd=True,
                          creators=[self],
                          creation_op="sum_"+str(dim))
        return Tensor(self.data.sum(dim))
    
    def expand(self, dim,copies):

        trans_cmd = list(range(0,len(self.data.shape)))
        trans_cmd.insert(dim,len(self.data.shape))
        new_data = self.data.repeat(copies).reshape(list(self.data.shape) + [copies]).transpose(trans_cmd)
        
        if(self.autograd):
            return Tensor(new_data,
                          autograd=True,
                          creators=[self],
                          creation_op="expand_"+str(dim))
        return Tensor(new_data)
    
    def transpose(self):
        if(self.autograd):
            return Tensor(self.data.transpose(),
                          autograd=True,
                          creators=[self],
                          creation_op="transpose")
        
        return Tensor(self.data.transpose())
    
    def mm(self, x):
        if(self.autograd):
            return Tensor(self.data.dot(x.data),
                          autograd=True,
                          creators=[self,x],
                          creation_op="mm")
        return Tensor(self.data.dot(x.data))
    
    def __repr__(self):
        return str(self.data.__repr__())
    
    def __str__(self):
        return str(self.data.__str__())  
    
    def cross_entropy(self, target_indices):

        temp = np.exp(self.data)
        softmax_output = temp / np.sum(temp,
                                       axis=len(self.data.shape)-1,
                                       keepdims=True)
        
        t = target_indices.data.flatten()
        p = softmax_output.reshape(len(t),-1)
        target_dist = np.eye(p.shape[1])[t]
        loss = -(np.log(p) * (target_dist)).sum(1).mean()
    
        if(self.autograd):
            out = Tensor(loss,
                         autograd=True,
                         creators=[self],
                         creation_op="cross_entropy")
            out.softmax_output = softmax_output
            out.target_dist = target_dist
            return out

        return Tensor(loss)
    
    def softmax(self):
        temp = np.exp(self.data)
        softmax_output = temp / np.sum(temp,
                                       axis=len(self.data.shape)-1,
                                       keepdims=True)
        return softmax_output
    
    def sigmoid(self):
        if self.autograd:
            return Tensor(1 / (1 + np.exp(- self.data)),
                          autograd=True,
                          creators=[self],
                          creation_op="sigmoid")
        return Tensor(1 / (1 + np.exp(- self.data)))
                      
    def tanh(self):
        if self.autograd:
            return Tensor(np.tanh(self.data),
                          autograd=True,
                          creators=[self],
                          creation_op="tanh")
        return Tensor(np.tanh(- self.data))
                      
    def relu(self):
        if self.autograd:
            return Tensor(((self.data > 0) * self.data),
                          autograd=True,
                          creators=[self],
                          creation_op="relu")
        return Tensor(((self.data > 0) * self.data))
    
    def index_select(self, indices):

        if(self.autograd):
            new = Tensor(self.data[indices.data],
                         autograd=True,
                         creators=[self],
                         creation_op="index_select")
            new.index_select_indices = indices
            return new
        return Tensor(self.data[indices.data])

    
    
class CrossEntropyLoss(object):
    
    def __init__(self):
        super().__init__()
    
    def forward(self, inp, target):
        return inp.cross_entropy(target)
    
class Layer():
    
    def __init__(self):
        self.params = []
    
    def get_params(self):
        return self.params

class Linear(Layer):
    
    def __init__(self, n_inputs, n_outputs):
        super().__init__()
        # np.random.randn возвращает массив нормально распределенных величин около 0
        # размера (n_inputs, n_outputs)
        W = np.random.randn(n_inputs, n_outputs) * np.sqrt(2.0/n_inputs)
        self.weight = Tensor(W, autograd=True)
        self.bias = Tensor(np.zeros(n_outputs), autograd=True)
        
        self.params.append(self.weight)
        self.params.append(self.bias)
        
    def forward(self, inp):
        return inp.mm(self.weight)+self.bias.expand(0, len(inp.data))

class Sequential(Layer):
    
    def __init__(self, layers=list()):
        super().__init__()
        
        self.layers = layers
        
    def add(self, layer):
        self.layers.append(layer)
        
    def forward(self, inp):
        for layer in self.layers:
            inp = layer.forward(inp)
        return inp
    
    def get_params(self):
        params = []
        for l in self.layers:
            params += l.get_params()
        return params
    
    
class RNNCell(Layer):
    
    def __init__(self, n_inputs, n_hidden, n_output, activation='sigmoid'):
        super().__init__()

        self.n_inputs = n_inputs
        self.n_hidden = n_hidden
        self.n_output = n_output
        
        if(activation == 'sigmoid'):
            self.activation = Sigmoid()
        elif(activation == 'tanh'):
            self.activation == Tanh()
        else:
            raise Exception("Non-linearity not found")

        self.w_ih = Linear(n_inputs, n_hidden)
        self.w_hh = Linear(n_hidden, n_hidden)
        self.w_ho = Linear(n_hidden, n_output)
        
        self.params += self.w_ih.get_params()
        self.params += self.w_hh.get_params()
        self.params += self.w_ho.get_params()        
    
    def forward(self, input, hidden):
        from_prev_hidden = self.w_hh.forward(hidden)
        combined = self.w_ih.forward(input) + from_prev_hidden
        new_hidden = self.activation.forward(combined)
        output = self.w_ho.forward(new_hidden)
        return output, new_hidden
    
    def init_hidden(self, batch_size=1):
        return Tensor(np.zeros((batch_size,self.n_hidden)), autograd=True)
    
class MSELoss(Layer):
    
    def __init__(self):
        super().__init__()
        
    def forward(self, pred, target):
        return ((pred - target)*(pred - target)).sum(0)
    
class Tanh(Layer):
    
    def __init__(self):
        super().__init__()
        
    def forward(self, inp):
        return inp.tanh()
    
    
class Sigmoid(Layer):
    def __init__(self):
        super().__init__()
        
    def forward(self, inp):
        return inp.sigmoid()
    
class Embedding(Layer):
    
    def __init__(self, vocab_size, dim):
        super().__init__()
        
        self.vocab_size = vocab_size
        self.dim = dim
        
        weight = (np.random.rand(vocab_size, dim) - 0.5) / dim
        self.weight = Tensor(weight, autograd=True)
        self.params.append(self.weight)
        
    def forward(self, inp):
        return self.weight.index_select(inp)
    
class SGD(object):
    
    def __init__(self, params, alpha=0.1):
        self.params = params
        self.alpha = alpha
        
    def zero(self):
        for p in self.params:
            p.data -= p.grad.data * self.alpha
            if zero:
                p.grad.data *= 0     
                
    def step(self, zero=True):
        for p in self.params:
            p.data -= p.grad.data * self.alpha
            
            if zero:
                p.grad.data *= 0

In [3]:
embed = Embedding(vocab_size=len(vocab),dim=512)
model = RNNCell(n_inputs=512, n_hidden=512, n_output=len(vocab))

criterion = CrossEntropyLoss()
optim = SGD(params=model.get_params() + embed.get_params(), alpha=0.05)

In [4]:
batch_size = 32
bptt = 16
n_batches = int((indices.shape[0] / batch_size))

In [5]:
trimmed_indices = indices[:n_batches * batch_size]
batched_indices = trimmed_indices.reshape(batch_size, n_batches)
batched_indices = batched_indices.T

input_batched_indices = batched_indices[0:-1]
target_batched_indices = batched_indices[1:]

n_bptt = int((n_batches-1)/bptt)
input_batches = input_batched_indices[:n_bptt*bptt]
input_batches = input_batches.reshape(n_bptt, bptt, batch_size)
target_batches = target_batched_indices[:n_bptt*bptt].reshape(n_bptt, bptt, batch_size)

In [6]:
print(raw[:5])
print(indices[:5])
print(batched_indices[:5])


Все 
[60 59 68 74 51]
[[60 51 21 51 29 28 23 51 22 85 51 74 93 92 68  3 29 51 68 29 80 74 51 68
  95 93 51 80 51 35 39 28]
 [59 21 35 94  1 10 74 28 68 51 21 94 51 80 39 68 30  3 68 21 21 21 29 93
  61 63 28 94 92 30 29 30]
 [68 35 10  1  3 74 40 51  3 41 29 80 43 74 79 43 51 35 39 56 10 30 21 51
  30 74 51 90 10 87 39 51]
 [74 41 74 63 51 51 74 80 29 80 92 21 51 20 51 30 43 51 29 60 80 51  3 39
  51 68 68 29 74 13 51 42]
 [51 28 68 28 29 10 68 63 21 51 28 74 21 80 83 51 51 21 62 60 85 41 80 94
  59 43 29  3 51 51 80 80]]


In [7]:
def generate_sample(n=30, init_char=' '):
    s = ""
    hidden = model.init_hidden(batch_size=1)
    inp = Tensor(np.array([word2index[init_char]]))
    for i in range(n):
        rnn_input = embed.forward(inp)
        output, hidden = model.forward(rnn_input, hidden)

        m = output.data.argmax()
        c = vocab[m]
        inp = Tensor(np.array([m]))
        s += c
    return s

def train(iterations=100):
    
    for iteration in range(iterations):
        total_loss = 0
        n_loss = 0

        hidden = model.init_hidden(batch_size=batch_size)
        batches_to_train = len(input_batches)
        for batch_i in range(batches_to_train):

            hidden = Tensor(hidden.data, autograd=True)
            loss = None
            losses = []
            for t in range(bptt):
                inp = Tensor(input_batches[batch_i][t], autograd=True)
                rnn_input = embed.forward(inp)
                output, hidden = model.forward(rnn_input, hidden)

                target = Tensor(target_batches[batch_i][t], autograd=True)    
                batch_loss = criterion.forward(output, target)
                losses.append(batch_loss)
                
                if(t == 0):
                    loss = batch_loss
#                     losses.append(batch_loss)
                else:
                    loss += batch_loss
#                     losses.append(batch_loss + losses[-1])

            for loss in losses:
                ''

            loss.backward()
            optim.step()
            total_loss += loss.data

            log = "\r Iter:" + str(iteration)
            log += " - Batch "+str(batch_i+1)+"/"+str(len(input_batches))
            log += " - Loss:" + str(np.exp(total_loss / (batch_i+1)))
            if(batch_i == 0):
                log += " - " + generate_sample(70, '\n').replace("\n"," ")
            if(batch_i % 10 == 0 or batch_i-1 == len(input_batches)):
                sys.stdout.write(log)
        optim.alpha *= 0.99
        print()

In [8]:
train()

 Iter:0 - Batch 91/100 - Loss:334.74618404861765-                                                                       
 Iter:1 - Batch 91/100 - Loss:31.010225517168447- о  а                                                                  
 Iter:2 - Batch 91/100 - Loss:24.110065888373743- о  с и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и
 Iter:3 - Batch 91/100 - Loss:21.568881866006735- о  о и и с с и и и и и и и и и и и и и и и и и и и и и и и и и и и и и
 Iter:4 - Batch 91/100 - Loss:19.666693376799497- о  о и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и
 Iter:5 - Batch 91/100 - Loss:18.325686052741688- рь о и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и
 Iter:6 - Batch 91/100 - Loss:17.244541899763634- рь о и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и
 Iter:7 - Batch 91/100 - Loss:16.405635403860753- рь о и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и и
 Iter:8 - Batch 91/100 - Loss:15

 Iter:67 - Batch 91/100 - Loss:1.2874395071333748 оло. Бо ум ум меня он минул с безза денерь меня он минул с безза денер
 Iter:68 - Batch 91/100 - Loss:1.2609297628509024 оло. Бо ум ум меня он мину онитрани ер, не ну тредов. Нух оди разут, к
 Iter:69 - Batch 91/100 - Loss:1.2371934313979282- оло. Бо ум ум меня он мину онитрани ер, не ну тредные глаза моет и пом
 Iter:70 - Batch 91/100 - Loss:1.2162139094021083- оло. Бо у скогда ериммоет это концы когда меня он мину онитрани ер, не
 Iter:71 - Batch 91/100 - Loss:1.1979551443946306- оло. Бо у скогдреет это мно… Нел с безза денецать этоймается, не выдо 
 Iter:72 - Batch 91/100 - Loss:1.1817968563697198- оло. Бо у скогдреет это мно… Нел с безза денецать этоймается, не выдош
 Iter:73 - Batch 91/100 - Loss:1.1685111415313436- оло. Бо у свою нассудия и помно… Нух оди разут, Ча не ну тредноет это 
 Iter:74 - Batch 91/100 - Loss:1.1553327403266658- оло. Бо у скогдреет это мно… Нел с безза денецать интеллдет ду ер, но 
 Iter:75 - Batch 91/100 - 

In [9]:
def generate_sample(n=30, init_char=' '):
    s = ""
    hidden = model.init_hidden(batch_size=1)
    inp = Tensor(np.array([word2index[init_char]]))
    for i in range(n):
        rnn_input = embed.forward(inp)
        output, hidden = model.forward(rnn_input, hidden)
        output.data *= 10
        temp_dist = output.softmax()
        temp_dist /= temp_dist.sum()
        
        m = (temp_dist > np.random.rand()).argmax()
        c = vocab[m]
        inp = Tensor(np.array([m]))
        s += c
    return s

print(generate_sample(n=2000, init_char='\n'))


ё крочил, а не стебе у развое толосто не занемёил, а не стебе у развое топрать – отчера моглай расский цет сам сумно… Не я – отчерь меня он меня они пповалдницу! ОНе чемное, или в ёпросто, о сто к только фуходиния денецать интеллы. Не не стебе у развое тоёё пет ду расский цет сам сумно… Не я – отчерь меня он меня они пповалдниё пет уж вся черезё всем за все сёет илается, нее, а то бы меня он меня они ппоё все выбожьше: Скажа меня он меня они пповёкая не сого свот минул с беззалдет сим с беззало выдоши сёет иё ператьсиё гразут, Чиремно… Нух оди, выдово
– Котом и гла и ёни приза божел! или разжи ёзя ду, а тоже бозя не выдоши сказали и пакак то гово
– О в чзогогда есть стаки ил размай расскогда же голосто не занемитривали измотаки ёни разжи ёзя был бы меня он меня они ппоё все выбожьше: Скажа меня он меня они ппово
– О в ца меня он меня они пповалдницу! ОНе чемное, или в впом умает, а ёечера моглай расскогда же голосто пы у не ду, а тоже бозли двед, лочемно… Не не сёет это кончёлов. Но н

In [10]:
print(generate_sample(n=2000, init_char=' '))

пихаёиё бзяж б было выдошил, а Семпом умает, а Семпоё все выёскончемай рассудёя, не выбодвою нассудия, но выдоши сказали интеллю илед, когда миоб бозмалёчелай взях одивалдет это, до всем за все выбожьше: ё встаю лы глаза не сто нед, я сам сумноет, я и сто кончерал. Но уна сто нед, я сам сумёиё в тоже бозли двед, лочемно… Не не стебе у развое толёчалосто не занемитривалиёский взяхёда меня он меня они ппот душе, две ного фе я – отчера моёбо нед, я – он было ни раё крода бодел!.. Не не ёыроши сто нед, я сам сумноет, я и сто кончёлов. Но нее, до верестолько фуходиния денецать интеллы. Не не стё не заресь отчера моглай расскёи в впом умает, а Семпой черелайтай черемносто нед, я сам сумноетё у это, а довёусто не занем и помподвою назаресная душик за грабо мою настеллёё взяходь когда же я отчал, а не стебе у развое топрать – отчера моглай расскогда ербово
– О вечайтельсках палосто не занемитривали измотаки меня он меня они ппово
– О в ца меня он меня они ппот душе, две ного фе я – отчера моёб